In [ ]:
import xml.etree.ElementTree as Xet
import pandas as pd

# *** takes more than 1 hour to run ***

with open('1.full database.xml', 'r', encoding="utf-8") as xml_file:
    xmlparse = Xet.parse(xml_file)

root = xmlparse.getroot()

In [ ]:
rows = []
n = 0
for drug in root:
    n += 1
    name = drug.find("{http://www.drugbank.ca}name").text
    drugbankid = drug.find("{http://www.drugbank.ca}drugbank-id").text
    atc_code_elements = drug.findall("{http://www.drugbank.ca}atc-codes/{http://www.drugbank.ca}atc-code")
    targets_elements = drug.findall("{http://www.drugbank.ca}targets/{http://www.drugbank.ca}target/{http://www.drugbank.ca}polypeptide")
    category_elements = drug.findall("{http://www.drugbank.ca}categories/{http://www.drugbank.ca}category")
    drug_interactions_elements = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction")
    #drug_interaction_drugs = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction/{http://www.drugbank.ca}name")
    #drug_interaction_descriptions = drug.findall("{http://www.drugbank.ca}drug-interactions/{http://www.drugbank.ca}drug-interaction/{http://www.drugbank.ca}description")
    enzymes_elements = drug.findall("{http://www.drugbank.ca}enzymes/{http://www.drugbank.ca}enzyme")
    #enzymes_uniportId = drug.findall("{http://www.drugbank.ca}enzymes/{http://www.drugbank.ca}enzyme/{http://www.drugbank.ca}polypeptide")
    # for extract smiles
    calculated_properties_elements = drug.findall("{http://www.drugbank.ca}calculated-properties/{http://www.drugbank.ca}property")
    #calculated_value = drug.findall("{http://www.drugbank.ca}calculated-properties/{http://www.drugbank.ca}property/{http://www.drugbank.ca}value")

    
    if len(name) == 0:
        continue
    print(name)
    
    if atc_code_elements is not None:
        atc_codes = "\""
        for atc_code_element in atc_code_elements:
            atc_code = atc_code_element.attrib["code"]
            if len(atc_codes) == 1:
                atc_codes += atc_code
            else:
                atc_codes += "\n" + atc_code
        atc_codes += "\""


    if targets_elements is not None:
        targets = "\""
        for target_element in targets_elements:
            trgt = target_element.attrib["id"]
            if len(targets) == 1:
                targets += trgt
            else:
                targets += "\n" + trgt
        targets += "\""

    categories = "\""
    if category_elements is not None:
        for category_element in category_elements:
            category = category_element.find("{http://www.drugbank.ca}category")
            meshid = category_element.find("{http://www.drugbank.ca}mesh-id")
            #print(category)
            #print(meshid)
            if category is None:
                continue
            else:
                category = category.text
            if meshid is None:
                meshid = ""
            else:
                if meshid.text is None:
                    meshid = ""
                else:
                    meshid = meshid.text
            #print(category)
            #print(meshid)
            if len(categories) == 1:
                categories += category + "/" + meshid
            else:
                categories += "\n" + category + "/" + meshid
    categories += "\""
                
    interactions = "\""
    if drug_interactions_elements is not None:
        for drug_interaction_element in drug_interactions_elements:
            idd = drug_interaction_element.find("{http://www.drugbank.ca}drugbank-id")
            interactionname = drug_interaction_element.find("{http://www.drugbank.ca}name")
            desc = drug_interaction_element.find("{http://www.drugbank.ca}description")
            if idd is None:
                continue
            else:
                idd = idd.text
            if interactionname is None:
                continue
            else:
                interactionname = interactionname.text
            if desc is None:
                desc = ""
            else:
                desc = desc.text
                
            if len(interactions) == 1:
                interactions += idd + "/ " + interactionname + "/ " + desc
            else:

                interactions += "\n" + idd + "/ " + interactionname + "/ " + desc
    interactions += "\""

    enzymes = "\""
    if enzymes_elements is not None:
        for enzyme_element in enzymes_elements:
            enzymename = enzyme_element.find("{http://www.drugbank.ca}name")
            idd = enzyme_element.find("{http://www.drugbank.ca}polypeptide")
            if enzymename is None:
                continue
            else:
                enzymename = enzymename.text
            if idd is None:
                idd = ""
            else:
                idd = idd.attrib["id"]
            if len(enzymes) == 1:
                enzymes += enzymename + "/" + idd
            else:
                enzymes += "\n" + enzymename + "/" + idd
    #print(enzymes)
    enzymes += "\""
    
    smiles = "\""
    if calculated_properties_elements is not None:
        for calculated_element in calculated_properties_elements:
            if calculated_element.find("{http://www.drugbank.ca}kind").text == "SMILES":
                smiles += calculated_element.find("{http://www.drugbank.ca}value").text
    smiles += "\""
    
    rows.append({"drug": name,
                 "drugbank.id": drugbankid,
                "atc-codes": atc_codes,
                "targets": targets,
                "categories": categories,
                "drug_interactions": interactions,
                "enzymes": enzymes,
                "smiles": smiles})
    
    if n % 100 == 0:
        print(n)
print("last one:", n)
del(xmlparse)
del(root)
print("***** making dataframe *****")
df = pd.DataFrame(rows, columns=["drug", "drugbank.id", "atc-codes", "targets", "categories", "drug_interactions", "enzymes", "smiles"])
print("****** writng to file *****")
df.to_csv("output.csv")



In [ ]:
# *********** extract xml nodes and save to file *****************
tree = Xet.ElementTree(root[5])
tree.write("test.xml", encoding="utf-8")